In [22]:
import numpy as np
import talib as ta
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LSTM, Dense, Input, concatenate, Layer, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [23]:
# ================== 增强版数据预处理 ==================
def preprocess_data(file_path, time_scales=['30min']):
    # 读取原始数据
    # df = pd.read_csv(file_path, parse_dates=['date'], index_col='date')
    
    # 技术指标计算
    def add_technical_indicators(df):
        # 价格特征
        # df['price_diff'] = df['close'].diff()
        # df['returns'] = df['close'].pct_change()
        
        # # 波动率指标
        # df['volatility_20'] = df['returns'].rolling(20).std()
        
        # # 技术指标
        # df['MA_10'] = df['close'].rolling(10).mean()
        # df['RSI'] = ta.RSI(df['close'], window=14)
        # df['MACD'], df['Signal'], df['Hist'] = ta.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
        return df.dropna()

    # 多尺度特征工程
    scaled_data = {}
    for scale in time_scales:
        # 重采样
        # 读取原始数据
        resampled = pd.read_csv(f"{file_path}/train-{scale}.csv", parse_dates=['date'], index_col='date')
        # resampled = df.resample(scale).agg({
        #     'open': 'first',
        #     'high': 'max',
        #     'low': 'min',
        #     'close': 'last',
        #     'volume': 'sum'
        # }).ffill()
        
        # 添加技术指标
        resampled = add_technical_indicators(resampled)
        
        # 归一化处理
        scaler = MinMaxScaler(feature_range=(0,1))
        scaled = scaler.fit_transform(resampled)
        
        # 构建3D时序数据 [samples, timesteps, features]
        seq_length = 60  # 使用60个时间窗口
        X, y = [], []
        for i in range(len(scaled)-seq_length-1):
            X.append(scaled[i:i+seq_length])
            y.append(scaled[i+seq_length, 3])  # 预测close价格
        scaled_data[scale] = (np.array(X), np.array(y))
    
    return scaled_data, scaler

In [24]:
# ================== MoE 模型架构 ==================
class ExpertNetwork(Layer):
    def __init__(self, units, **kwargs):
        super(ExpertNetwork, self).__init__(**kwargs)
        self.lstm1 = LSTM(units[0], return_sequences=True)
        self.lstm2 = LSTM(units[1])
        self.dense = Dense(1)

    def call(self, inputs):
        x = self.lstm1(inputs)
        x = self.lstm2(x)
        return self.dense(x)

class MoE(Model):
    def __init__(self, num_experts, time_scales, **kwargs):
        super(MoE, self).__init__(**kwargs)
        self.experts = [ExpertNetwork([64,32]) for _ in range(num_experts)]
        self.attention = Attention(use_scale=True)
        self.gate = Dense(num_experts, activation='softmax')
        self.final_dense = Dense(1)
        self.time_scales = time_scales

    def call(self, inputs):
        # 专家输出
        expert_outputs = []
        for i, expert in enumerate(self.experts):
            scale_input = inputs[:,i,:,:]  # 每个专家处理对应尺度数据
            expert_outputs.append(expert(scale_input))
        
        # 动态路由
        concatenated = concatenate(expert_outputs)
        attention_weights = self.attention([concatenated, concatenated])
        gated_output = tf.reduce_sum(attention_weights * concatenated, axis=1)
        
        # 门控融合
        gate_weights = self.gate(gated_output)
        weighted_output = tf.reduce_sum(gate_weights * concatenated, axis=1)
        
        return self.final_dense(weighted_output)

# ================== 模型训练 ==================
def train_moe_model(data_dict):
    # 初始化模型
    model = MoE(num_experts=len(data_dict), time_scales=data_dict.keys())
    
    # 准备多尺度输入数据
    X_list = [data[0] for data in data_dict.values()]
    y = list(data_dict.values())[0][1]  # 假设所有尺度使用相同目标
    
    # 模型编译
    model.compile(optimizer='adamax',
                loss='mse',
                metrics=['mae'])
    model.summary()
    # 早停策略
    es = EarlyStopping(monitor='val_loss', patience=5)
    
    # 模型训练
    history = model.fit(X_list, y,
                      epochs=100,
                      batch_size=64,
                      validation_split=0.2,
                      callbacks=[es])
    print("预测模型...")
    return model

In [25]:
# ================== 模型预测与评估 ==================
def evaluate_moe(model, test_file, scaler, time_scales=['30min']):
    # 加载测试数据
    test_data, _ = preprocess_data(test_file, time_scales)
    
    # 准备多尺度测试输入
    X_test_list = [data[0] for data in test_data.values()]
    y_test = list(test_data.values())[0][1]

    # 模型预测
    predictions = model.predict(X_test_list)
    
    # 反归一化处理
    def inverse_scale(data, scaler, feature_index=3):
        dummy = np.zeros(shape=(len(data), scaler.n_features_in_))
        dummy[:, feature_index] = data
        return scaler.inverse_transform(dummy)[:, feature_index]

    # 获取实际价格序列
    real_prices = inverse_scale(y_test, scaler)
    predicted_prices = inverse_scale(predictions, scaler)

    # 计算评估指标
    mse = mean_squared_error(real_prices, predicted_prices)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(real_prices, predicted_prices)
    r2 = r2_score(real_prices, predicted_prices)

    # 可视化对比
    plt.figure(figsize=(15, 6))
    plt.plot(real_prices, label='Actual Prices', alpha=0.7)
    plt.plot(predicted_prices, label='Predicted Prices', linestyle='--')
    plt.title(f'MoE Model Prediction Results\nMSE: {mse:.4f}  MAE: {mae:.4f}  R²: {r2:.4f}')
    plt.legend()
    plt.grid(True)
    plt.savefig('moe_prediction_comparison.png')
    plt.show()

    # 返回指标字典
    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'Predictions': predicted_prices,
        'Actuals': real_prices
    }

In [26]:
# 数据预处理
data_scaled, scaler = preprocess_data('../data/')

# 模型训练
moe_model = train_moe_model(data_scaled)

# 模型保存
moe_model.save('moe_stock_predictor.h5')

Model: "mo_e"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ expert_network (ExpertNetwork)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention (Attention)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100


c:\Users\liufa\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\keras\src\layers\layer.py:1407: UserWarning: Layer 'mo_e' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''tuple indices must be integers or slices, not tuple''
  warnings.warn(
c:\Users\liufa\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\keras\src\layers\layer.py:395: UserWarning: `build()` was called on layer 'mo_e', however the layer does not have a `build()` method implemented and it 

TypeError: Exception encountered when calling MoE.call().

[1mtuple indices must be integers or slices, not tuple[0m

Arguments received by MoE.call():
  • inputs=('tf.Tensor(shape=(None, 60, 31), dtype=float32)',)